In [1]:
# Standard library imports
import sys
import os
import requests
import itertools
import math
from io import StringIO
from tqdm import tqdm
from typing import Tuple

# 3rd-party library imports
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Step 0

In [2]:
'''
Step 0: Downloading Data

Combining diverse inputs into a single dataset

Inputs include:
    - GHCN v4 data
    - ERRST v5 data (later on?)
'''

# Standard library imports
import requests
import sys
import os
from typing import List

# 3rd-party library imports
import pandas as pd
import numpy as np

# Add the parent folder to sys.path
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.insert(0, parent_dir)

# Local imports
# from parameters.data import GHCN_temp_url, GHCN_meta_url
GHCN_temp_url = 'https://data.giss.nasa.gov/pub/gistemp/ghcnm.tavg.qcf.dat'
GHCN_meta_url = 'https://data.giss.nasa.gov/pub/gistemp/v4.inv'

# Local imports
from parameters.data import GHCN_temp_url, GHCN_meta_url

def get_GHCN_data(temp_url: str, meta_url: str) -> pd.DataFrame:
    '''
    Retrieves and formats temperature data from the Global Historical Climatology Network (GHCN) dataset.

    Args:
    temp_url (str): The URL to the temperature data file in GHCN format.
    meta_url (str): The URL to the metadata file containing station information.

    Returns:
    pd.DataFrame: A Pandas DataFrame containing temperature data with station metadata.
    
    This function sends an HTTP GET request to the temperature data URL, processes the data to create
    a formatted DataFrame, replaces missing values with NaN, converts temperature values to degrees Celsius,
    and merges the data with station metadata based on station IDs. The resulting DataFrame includes
    columns for station latitude, longitude, and name, and is indexed by station IDs.
    '''

    try:
        # Send an HTTP GET request to the URL
        response = requests.get(temp_url)

        # Check if the request was successful
        if response.status_code == 200:
            
            # Get the content of the response
            file_data: str = response.content.decode("utf-8")

            # Create a list to store formatted data
            formatted_data = []

            # Loop through file data
            for line in file_data.split('\n'):
                
                # Check if line is not empty
                if line.strip():
                    
                    # Extract relevant data
                    # (Using code from GHCNV4Reader())
                    station_id: str = line[:11]
                    year: int = int(line[11:15])
                    values: List[int] = [int(line[i:i+5]) for i in range(19, 115, 8)]
                    
                    # Append data to list
                    formatted_data.append([station_id, year] + values)

            # Create DataFrame from formatted data
            column_names: List[str] = ['Station_ID', 'Year'] + [f'Month_{i}' for i in range(1, 13)]
            df_GHCN: pd.DataFrame = pd.DataFrame(formatted_data, columns=column_names)
            
            # Replace -9999 with NaN
            df_GHCN.replace(-9999, np.nan, inplace=True)
            
            # Format data - convert to degrees C
            month_columns: List[str] = [f'Month_{i}' for i in range(1, 13)]
            df_GHCN[month_columns] = df_GHCN[month_columns].divide(100)

        else:
            print("Failed to download the file. Status code:", response.status_code)

    except Exception as e:
        print("An error occurred:", str(e))

    # Define the column widths, create meta data dataframe
    column_widths: List[int] = [11, 9, 10, 7, 3, 31]
    df_meta: pd.DataFrame = pd.read_fwf(meta_url, widths=column_widths, header=None,
                          names=['Station_ID', 'Latitude', 'Longitude', 'Elevation', 'State', 'Name'])
    # Merge on station ID, set index
    df: pd.DataFrame = pd.merge(df_GHCN, df_meta[['Station_ID', 'Latitude', 'Longitude', 'Name']], on='Station_ID', how='left')
    df = df.set_index('Station_ID')

    return df

def step0() -> pd.DataFrame:
    '''
    Performs the initial data processing steps for the GHCN temperature dataset.

    Returns:
    pd.DataFrame: A Pandas DataFrame containing filtered and formatted temperature data.
    
    This function retrieves temperature data from the Global Historical Climatology Network (GHCN) dataset,
    processes and formats the data, and returns a DataFrame. The data is first fetched using specified URLs,
    and is returned for further analysis.
    '''
    df_GHCN: pd.DataFrame = get_GHCN_data(GHCN_temp_url, GHCN_meta_url)
    return df_GHCN

In [3]:
step0_output = step0()

# Step 1

In [4]:
'''
Step 1: Removal of bad data

Drop or adjust certain records (or parts of records).
This includes outliers / out of range reports.
Determined using configuration file.
    <TO-DO> Figure out if this method is ideal.
'''

import pandas as pd
import os
import re
import sys

# Add the parent folder to sys.path
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.insert(0, parent_dir)

# Local imports
# from parameters.data import drop_rules
drop_rules = '''
CHM00052836  omit: 0-1948
CHXLT909860  omit: 0-1950
BL000085365  omit: 0-1930
MXXLT948335  omit: 0-1952
ASN00058012  omit: 0-1899
ASN00084016  omit: 0-1899
ASN00069018  omit: 0-1898
NIXLT013080  omit: 0-1930
NIXLT751359  omit: 0-9999
CHXLT063941  omit: 0-1937
CHM00054843  omit: 0-1937
MXM00076373  omit: 0-9999
USC00044022  omit: 0-9999
USC00044025  omit: 0-9999
CA002402332  omit: 2011-9999
RSM00024266  omit: 2021/09
'''


def filter_coordinates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filters a DataFrame based on latitude and longitude conditions.

    Args:
    df (pd.DataFrame): The input DataFrame with 'Latitude' and 'Longitude' columns.

    Returns:
    pd.DataFrame: The filtered DataFrame with rows where latitude is between -90 and 90,
    and longitude is between -180 and 180.
    """
    
    # Define latitude and longitude range conditions
    lat_condition = (df['Latitude'] >= -90) & (df['Latitude'] <= 90)
    lon_condition = (df['Longitude'] >= -180) & (df['Longitude'] <= 180)

    # Apply the conditions to filter the DataFrame
    df_filtered = df[lat_condition & lon_condition]
    
    # Calculate number of rows filtered
    num_filtered = len(df) - len(df_filtered)
    print(f'Number of rows with invalid coordinates (removed): {num_filtered}')

    return df_filtered

def filter_stations_by_rules(dataframe: pd.DataFrame, rules_text: str) -> pd.DataFrame:
    """
    Filters a DataFrame of climate station data based on exclusion rules specified in a text format.

    Parameters:
        dataframe (pd.DataFrame): The input DataFrame containing climate station data.
        rules_text (str): A string containing exclusion rules for specific stations and years.

    Returns:
        pd.DataFrame: A filtered DataFrame with stations omitted based on the provided rules.

    Rules Format:
        The 'rules_text' should be formatted as follows:
        - Each rule is represented as a single line in the text.
        - Each line should start with the station ID followed by exclusion rules.
        - Exclusion rules consist of 'omit:' followed by the years to exclude, e.g., 'omit: 2000-2010'.
        - Years can be specified as a single year (e.g., 'omit: 2000') or as a range (e.g., 'omit: 2000-2010').
        - Year ranges can also be specified using '/' (e.g., 'omit: 2000/2002').

    Example:
        rules_text = '''
            CHM00052836  omit: 0-1948
            CHXLT909860  omit: 0-1950
            BL000085365  omit: 0-1930
            ...
        '''

    This function takes the provided rules and applies them to the input DataFrame,
    resulting in a new DataFrame with stations excluded based on the specified rules.
    """

    # Parse the rules from the provided text
    rules = {}
    for line in rules_text.split('\n'):
        if line.strip():
            match = re.match(r'([A-Z0-9]+)\s+omit:\s+(\S+)', line)
            if match:
                station_id, year_rule = match.groups()
                rules[station_id] = year_rule

    # Create a mask to identify rows to omit
    mask = pd.Series(True, index=dataframe.index)

    for station_id, year_rule in rules.items():
        try:
            # Split the year_rule into start and end years
            start_year, end_year = map(int, year_rule.split('-'))
        except ValueError:
            # Handle cases like '2011/12' or '2012-9999'
            if '/' in year_rule:
                start_year = int(year_rule.split('/')[0])
                end_year = start_year
            elif '-' in year_rule:
                start_year = int(year_rule.split('-')[0])
                end_year = int(year_rule.split('-')[1])
            else:
                continue

        # Update the mask to False for the specified range of years for the station_id
        mask &= ~((dataframe['Year'] >= start_year) & (dataframe['Year'] <= end_year) & (dataframe.index == station_id))

    # Apply the mask to filter the DataFrame
    filtered_dataframe = dataframe[mask]

    # Calculate number of rows filtered
    num_filtered = len(dataframe) - len(filtered_dataframe)
    print(f'Number of rows removed according to station exclusion rules: {num_filtered}')

    return filtered_dataframe

def step1(step0_output: pd.DataFrame) -> pd.DataFrame:
    """
    Applies data filtering and cleaning operations to the input DataFrame.

    Parameters:
        step0_output (pd.DataFrame): The initial DataFrame containing climate station data.

    Returns:
        pd.DataFrame: A cleaned and filtered DataFrame ready for further analysis.

    This function serves as a data processing step by applying two essential filtering operations:
    1. `filter_coordinates`: Filters the DataFrame based on geographical coordinates, retaining relevant stations.
    2. `filter_stations_by_rules`: Filters the DataFrame based on exclusion rules, omitting specified stations and years.

    The resulting DataFrame is cleaned of irrelevant stations and years according to specified rules
    and is ready for subsequent data analysis or visualization.
    """
        
    df_filtered = filter_coordinates(step0_output)
    df_clean = filter_stations_by_rules(df_filtered, drop_rules)
    return df_clean

In [5]:
step1_output = step1(step0_output)

Number of rows with invalid coordinates (removed): 0
Number of rows removed according to station exclusion rules: 524


# Step 2

In [6]:
# Skip for now

# Step 3

In [37]:
'''
Step 3: Gridding of cells

There are 8000 cells across the globe.
Each cell's values are computed using station records within a 1200km radius.
    - Contributions are weighted according to distance to cell center
    (linearly decreasing to 0 at distance 1200km)
'''

import math
from typing import Tuple

import numpy as np
import pandas as pd
from pandas import Series


def calculate_area(row: Series) -> float:
    earth_radius_km: float = 6371.0
    delta_longitude: float = np.radians(row['Eastern'] - row['Western'])
    southern_latitude: float = np.radians(row['Southern'])
    northern_latitude: float = np.radians(row['Northern'])
    area: float = (earth_radius_km ** 2) * delta_longitude * (np.sin(northern_latitude) - np.sin(southern_latitude))
    return area


def calculate_center_coordinates(row: pd.Series) -> Tuple[float, float]:
    """Calculate the center latitude and longitude for a given box.

    Args:
        row (pd.Series): A Pandas Series representing a row of the DataFrame with ('southern', 'northern', 'western', 'eastern') coordinates.

    Returns:
        Tuple[float, float]: A tuple containing the center latitude and longitude.
    """
    center_latitude = 0.5 * (math.sin(row['Southern'] * math.pi / 180) + math.sin(row['Northern'] * math.pi / 180))
    center_longitude = 0.5 * (row['Western'] + row['Eastern'])
    center_latitude = math.asin(center_latitude) * 180 / math.pi
    return center_latitude, center_longitude


def generate_80_cell_grid() -> pd.DataFrame:
    """Generate an 80-cell grid DataFrame with columns for southern, northern, western, eastern,
    center_latitude, and center_longitude coordinates.

    Returns:
        pd.DataFrame: The generated DataFrame.
    """
    grid_data = []
    
    # Number of horizontal boxes in each band
    # (proportional to the thickness of each band)
    band_boxes = [4, 8, 12, 16]
    
    # Sines of latitudes
    band_altitude = [1, 0.9, 0.7, 0.4, 0]

    # Generate the 40 cells in the northern hemisphere
    for band in range(len(band_boxes)):
        n = band_boxes[band]
        for i in range(n):
            lats = 180 / math.pi * math.asin(band_altitude[band + 1])
            latn = 180 / math.pi * math.asin(band_altitude[band])
            lonw = -180 + 360 * float(i) / n
            lone = -180 + 360 * float(i + 1) / n
            box = (lats, latn, lonw, lone)
            grid_data.append(box)

    # Generate the 40 cells in the southern hemisphere by reversing the northern hemisphere cells
    for box in grid_data[::-1]:
        grid_data.append((-box[1], -box[0], box[2], box[3]))

    # Create a DataFrame from the grid data
    df = pd.DataFrame(grid_data, columns=['Southern', 'Northern', 'Western', 'Eastern'])

    # Calculate center coordinates for each box and add them as new columns
    center_coords = df.apply(calculate_center_coordinates, axis=1)
    df[['Center_Latitude', 'Center_Longitude']] = pd.DataFrame(center_coords.tolist(), index=df.index)

    return df
    

def interpolate(x: float, y: float, p: float) -> float:
    return y * p + (1 - p) * x


def generate_8000_cell_grid(grid_80):

    # Initialize an empty list to store subboxes
    subbox_list = []

    for index, row in grid_80.iterrows():
        alts = math.sin(row['Southern'] * math.pi / 180)
        altn = math.sin(row['Northern'] * math.pi / 180)

        for y in range(10):
            s = 180 * math.asin(interpolate(alts, altn, y * 0.1)) / math.pi
            n = 180 * math.asin(interpolate(alts, altn, (y + 1) * 0.1)) / math.pi
            for x in range(10):
                w = interpolate(row['Western'], row['Eastern'], x * 0.1)
                e = interpolate(row['Western'], row['Eastern'], (x + 1) * 0.1)

                # Create a DataFrame for the subbox
                subbox_df = pd.DataFrame({'Southern': [s], 'Northern': [n], 'Western': [w], 'Eastern': [e]})

                # Append the subbox DataFrame to the list
                subbox_list.append(subbox_df)

    # Concatenate all subboxes into a single DataFrame
    grid_8000 = pd.concat(subbox_list, ignore_index=True)

    # Calculate center coordinates for each box and add them as new columns
    center_coords = grid_8000.apply(calculate_center_coordinates, axis=1)
    grid_8000[['Center_Latitude', 'Center_Longitude']] = pd.DataFrame(center_coords.tolist(), index=grid_8000.index)

    # Calculate area of all 8000 cells
    grid_8000['Area'] = grid_8000.apply(calculate_area, axis=1)

    # Print the resulting DataFrame
    return grid_8000

def haversine_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Calculate the spherical distance (in kilometers) between two pairs of
    latitude and longitude coordinates using the Haversine formula.

    Args:
        lat1 (float): Latitude of the first point in degrees.
        lon1 (float): Longitude of the first point in degrees.
        lat2 (float): Latitude of the second point in degrees.
        lon2 (float): Longitude of the second point in degrees.

    Returns:
        float: Spherical distance in kilometers.
    """
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Radius of the Earth in kilometers
    radius: float = 6371.0  # Earth's mean radius

    # Haversine formula
    dlat: float = lat2 - lat1
    dlon: float = lon2 - lon1

    a: float = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c: float = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance: float = radius * c

    return distance


def linearly_decreasing_weight(distance: float, max_distance: float) -> float:
    """
    Calculate a linearly decreasing weight based on the given distance
    and maximum distance.

    Args:
        distance (float): The distance at which you want to calculate the weight.
        max_distance (float): The maximum distance at which the weight becomes 0.

    Returns:
        float: The linearly decreasing weight, ranging from 1 to 0.
    """
    # Ensure that distance is within the valid range [0, max_distance]
    distance: float = max(0, min(distance, max_distance))

    # Calculate the weight as a linear interpolation
    weight: float = 1.0 - (distance / max_distance)
    
    return weight

def nearby_stations(grid_df):

    # Initialize an empty list to store station IDs and weights as dictionaries
    station_weights_within_radius = []

    # Maximum distance for the weight calculation (e.g., 1200.0 km)
    max_distance = 1200.0

    # Use tqdm to track progress
    for index, row in tqdm(grid_df.iterrows(), total=len(grid_df), desc="Processing"):
        center_lat = row['Center_Latitude']
        center_lon = row['Center_Longitude']

        # Calculate distances for each station in station_df
        distances = station_df.apply(lambda x: haversine_distance(center_lat, center_lon, x['Latitude'], x['Longitude']), axis=1)

        # Find station IDs within the specified radius
        nearby_stations = station_df[distances <= max_distance]

        # Calculate weights for each nearby station
        weights = nearby_stations.apply(lambda x: linearly_decreasing_weight(distances[x.name], max_distance), axis=1)

        # Create a dictionary of station IDs and weights
        station_weights = dict(zip(nearby_stations['Station_ID'], weights))

        # Append the dictionary to the result list
        station_weights_within_radius.append(station_weights)

    # Add the list of station IDs and weights as a new column
    grid_df['Nearby_Stations'] = station_weights_within_radius

    # Set index name
    grid_df.index.name = 'Box_Number'
    
    return grid_df

In [18]:
grid_80 = generate_80_cell_grid()
grid_80['Area'] = grid_80.apply(calculate_area, axis=1)

grid_8000 = generate_8000_cell_grid(grid_80)
grid_8000['Area'] = grid_8000.apply(calculate_area, axis=1)

In [23]:
meta_url = 'https://data.giss.nasa.gov/pub/gistemp/v4.inv'
column_widths: List[int] = [11, 9, 10, 7, 3, 31]
station_df: pd.DataFrame = pd.read_fwf(meta_url, widths=column_widths, header=None,
                          names=['Station_ID', 'Latitude', 'Longitude', 'Elevation', 'State', 'Name'])

In [33]:
grid_80 = nearby_stations(grid_80)

Processing: 100%|███████████████████████████████| 80/80 [00:06<00:00, 12.41it/s]


In [34]:
grid_8000 = nearby_stations(grid_8000)

Processing: 100%|███████████████████████████| 8000/8000 [10:41<00:00, 12.48it/s]


In [40]:
grid_8000

,Southern,Northern,Western,Eastern,Center_Latitude,Center_Longitude,Area,Nearby_Stations
Box_Number,,,,,,,,
0,64.158067,65.505352,-180.0,-171.0,64.823283,-175.5,63758.058989,"{'RSM00021965': 0.06629837649322723, 'RSM00021..."
1,64.158067,65.505352,-171.0,-162.0,64.823283,-166.5,63758.058989,"{'CA002100697': 0.003362114448123288, 'RSM0002..."
2,64.158067,65.505352,-162.0,-153.0,64.823283,-157.5,63758.058989,"{'CA002100100': 0.11900842694943892, 'CA002100..."
3,64.158067,65.505352,-153.0,-144.0,64.823283,-148.5,63758.058989,"{'CA001191440': 0.045975337634871716, 'CA00119..."
4,64.158067,65.505352,-144.0,-135.0,64.823283,-139.5,63758.058989,"{'CA001060330': 0.03890593070854176, 'CA001060..."
...,...,...,...,...,...,...,...,...
7995,-65.505352,-64.158067,-135.0,-126.0,-64.823283,-130.5,63758.058989,"{'AYM00089327': 0.21571371266640837, 'XXXLT848..."
7996,-65.505352,-64.158067,-126.0,-117.0,-64.823283,-121.5,63758.058989,"{'AYM00089327': 0.20306151170328113, 'XXXLT848..."
7997,-65.505352,-64.158067,-117.0,-108.0,-64.823283,-112.5,63758.058989,"{'AYM00089327': 0.09129442167304602, 'XXXLT848..."


In [57]:
def find_box_number(station_df, grid_80_df):
    box_numbers = []

    for _, station_row in tqdm(station_df.iterrows(), total=len(station_df)):
        latitude = station_row['Latitude']
        longitude = station_row['Longitude']

        for box_number, box_row in grid_80_df.iterrows():
            southern = box_row['Southern']
            northern = box_row['Northern']
            western = box_row['Western']
            eastern = box_row['Eastern']

            if southern <= latitude <= northern and western <= longitude <= eastern:
                box_numbers.append(box_number)
                break
        else:
            box_numbers.append(None)

    return box_numbers

In [58]:
# Find box numbers for each station, add to station_df
box_numbers = find_box_number(station_df, grid_80_df)
station_df['Box_Number'] = box_numbers

100%|█████████████████████████████████| 124954/124954 [01:19<00:00, 1570.55it/s]


In [59]:
station_df

,Station_ID,Latitude,Longitude,Elevation,State,Name,Box_Number
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,43
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,43
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,56
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,56
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,56
...,...,...,...,...,...,...,...
124949,ZI000067969,-21.0500,29.3670,861.0,NaN,WEST NICHOLSON,35
124950,ZI000067975,-20.0670,30.8670,1095.0,NaN,MASVINGO,35
124951,ZI000067977,-21.0170,31.5830,430.0,NaN,BUFFALO RANGE,35
124952,ZI000067983,-20.2000,32.6160,1132.0,NaN,CHIPINGE,35


# Step 4: SST Data

In [60]:
# Skipping for now

# Step 5: Anomalyzing Data

In [61]:
def anomalize_temperature_data(data, reference_period=(1951, 1980)):
    # Extract the years from the DataFrame
    years = data['Year'].unique()
    
    # Calculate monthly means for the reference period
    reference_data = data[(data['Year'] >= reference_period[0]) & (data['Year'] <= reference_period[1])]
    monthly_means = reference_data.iloc[:, 1:13].mean()
    
    # Initialize a DataFrame to store the anomalized data
    anomalized_data = data.copy()
    
    # Anomalize each month's data
    for month in tqdm(range(1, 13), desc="Anomalizing Months"):
        
        # Calculate the anomaly for the current month
        anomalized_data[f'Month_{month}'] = data.apply(lambda row: row[f'Month_{month}'] - monthly_means[month - 1], axis=1)
    
    return anomalized_data

In [62]:
df = step1_output
df_anom = anomalize_temperature_data(df, reference_period=(1951, 1980))
df_anom

Anomalizing Months: 100%|███████████████████████| 12/12 [00:44<00:00,  3.67s/it]


,Year,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Latitude,Longitude,Name
Station_ID,,,,,,,,,,,,,,,,
ACW00011604,1961,-1.439347,0.135949,-1.07872,-2.841062,-3.591227,-2.476789,-4.786892,-5.059991,-2.578559,-0.337097,-1.52523,-2.780427,57.7667,11.8667,VE
ACW00011604,1962,0.580653,-1.374051,-7.33872,-4.221062,-5.791227,-4.656789,-5.386892,-5.939991,-5.078559,-2.137097,-3.39523,-3.650427,57.7667,11.8667,VE
ACW00011604,1963,-7.679347,-7.754051,-6.78872,-5.161062,-2.631227,-2.196789,-4.286892,-3.909991,-3.388559,-2.677097,-0.96523,-3.470427,57.7667,11.8667,VE
ACW00011604,1964,0.070653,-3.074051,-5.24872,-3.191062,-2.681227,-4.046789,-5.426892,-4.299991,-4.498559,-4.197097,-1.16523,-1.270427,57.7667,11.8667,VE
ACW00011604,1965,-0.109347,-3.274051,-5.41872,-4.671062,-5.001227,-3.466789,-5.616892,-5.099991,-2.938559,-2.337097,-6.31523,-4.170427,57.7667,11.8667,VE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXLT622116,1966,21.290653,18.215949,12.64128,6.368938,-0.531227,-5.626789,-8.346892,-5.229991,1.031441,7.762903,14.31477,18.749573,-19.4300,29.7500,ELO
ZIXLT622116,1967,20.590653,17.715949,13.14128,8.668938,0.268773,-4.926789,-9.446892,-6.029991,-0.068559,8.762903,13.31477,16.749573,-19.4300,29.7500,ELO
ZIXLT622116,1968,21.290653,17.815949,13.54128,7.668938,0.768773,-7.626789,-6.746892,-3.529991,0.931441,9.762903,11.81477,18.349573,-19.4300,29.7500,ELO
